In [ ]:
import numpy as np
import scipy.stats as stats
from hansolo import *

# np.random.seed(10)

Nb = 100
expert = "EWA"  # "EWA" or "PWA"
learning="random" # "blocs": objects presented by random blocs, or "random": each object is randomly chosen with replacement


M = 2502
M_obj = 500 #size of testing set
n_epochs = int(M/9) #number of epochs
dt = 0.002
N = 1000
T = N * dt

freq = 100  # firing rate input neurons +
freq2 = 150  # firing rate input neurons -

p = freq * dt  # spiking proba input neurons
p2 = freq2 * dt

K_input = 12  # nb input neurons
K_output = 2  # nb output neurons


obj = [
    [1, 0, 0, 1, 0, 0],
    [1, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 1],
    [0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 0, 1, 0],
    [0, 0, 1, 0, 0, 1],
]
# blue circle blue square blue triangle red circle red square red triangle gray circle gray square gray triangle


activity = []  # contains active neurons for corresponding object

answers = np.zeros((Nb, n_epochs, M_obj))  # 0 if wrong, 1 if correct
W_output = np.load(f'/Users/sophiejaffard/Desktop/saves/{expert}_{learning}_W.npy')
#W_output = np.load(f'saves/{expert}_{learning}_W.npy')
W_output=W_output[:,:,9*np.arange(int(M/9)),:] #weights after each epoch

activity = [o + [1 - l for l in o] for o in obj]


for nb in range(Nb):
    #creation of the list of objects (testing set)
    L=list_objects_random(activity,M_obj)


    F_input = np.zeros((K_input, n_epochs, M_obj))
    F_output=np.zeros((K_output, n_epochs, M_obj))


    for m_obj in range(M_obj):

        activity_cur = L[m_obj]  # current active neurons
        obj_cur = activity_cur[: int(K_input / 2)]  # current object

        input_neurons=np.zeros((K_input, n_epochs, N))
        cur_act = np.where(activity_cur == 1)[0]
        first_split = cur_act[cur_act < (K_input // 2)]
        second_split = cur_act[cur_act >= (K_input // 2)]
        input_neurons[first_split, :, :] = Poisson(freq, T, (first_split.size, n_epochs, N))
        input_neurons[second_split, :, :] = Poisson(freq2, T, (second_split.size, n_epochs, N))

        output_neurons=np.zeros((K_output, n_epochs, N))

        # simulation of the output neurons
        
        for m in range(n_epochs):
            for i in range(K_output):
                output_neurons[i, m, :] = Hawkes_lin(input_neurons[:,m,:], W_output[i, :, m, nb])

        F_input[:, :, m_obj] = np.sum(input_neurons, axis=2) / T
        F_output[:, :, m_obj] = np.sum(output_neurons, axis=2) / T

        for m in range(n_epochs):
            if (in_A(obj_cur) and F_output[0, m, m_obj] > F_output[1, m, m_obj]) or (
                in_B(obj_cur) and F_output[1, m, m_obj] > F_output[0, m, m_obj]
            ):
                answers[nb, m, m_obj] = 1
        if m_obj%100==0:
            print(nb,m_obj)


np.save(f"{expert}_{learning}_test", answers)


